In [24]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer, MultiLabelBinarizer, scale, minmax_scale
from scipy.spatial.distance import cosine
from scipy import sparse
import lightgbm as lgb
%matplotlib inline

In [46]:
data_df = pd.read_csv('data_df_new.csv',index_col=0)

/home/sc3/miniconda2/envs/ydzhang/lib/python3.6/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
data_df[['user_id','question_id']].groupby('user_id').count().describe()

In [52]:
train_invite_files = [
                      'invite_info_[3840, 3860]-[3861, 3867]_quest-hist_topics_quest-user_follow_topics_quest-inn_topics_dist.csv',
#                      'invite_info_[3840, 3860]-[3861, 3867]_quest_hist_meanpool_dist.csv', 
#                      'invite_info_[3840, 3860]-[3861, 3867]_quest_hist_minpool_dist.csv',
#                     'invite_info_[3840, 3860]-[3861, 3867]_quest_hist_maxpool_dist.csv',
                    ]
test_invite_files = [
                     'invite_info_test-[3847, 3867]-[3868, 3874]_quest-hist_topics_quest-user_follow_topics_quest-inn_topics_dist.csv',
#                     'invite_info_test-[3847, 3867]-[3868, 3874]_quest_hist_maxpool_dist.csv',
#                     'invite_info_test-[3847, 3867]-[3868, 3874]_quest_hist_meanpool_dist.csv',
#                     'invite_info_test-[3847, 3867]-[3868, 3874]_quest_hist_minpool_dist.csv',
]

train_base_df = pd.read_csv('../ydzhang/data/'+train_invite_files[0],sep='\t',index_col=0)

test_base_df = pd.read_csv('../ydzhang/data/'+test_invite_files[0],sep='\t',index_col=0)
length = len(train_invite_files)
for i in range(1,length):
    train_t = pd.read_csv('../ydzhang/data/'+train_invite_files[i],sep='\t',index_col=0)
    train_base_df = pd.concat([train_base_df,train_t],axis = 1)
for i in range(1,length):
    test_t = pd.read_csv('../ydzhang/data/'+test_invite_files[i],sep='\t',index_col=0)
    test_base_df = pd.concat([test_base_df,test_t],axis = 1)

/home/sc3/miniconda2/envs/ydzhang/lib/python3.6/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [53]:
base_df = pd.concat([train_base_df,test_base_df],sort=False)

In [54]:
data_df = pd.concat([data_df,base_df],axis=1)

In [58]:
# data_df.drop(columns = base_df.columns,inplace=True)

In [55]:
for sec in base_df.columns:
    data_df[sec] = minmax_scale(data_df[sec])

In [25]:
test_df =pd.read_csv('evalute.csv',index_col=0)


/home/sc3/miniconda2/envs/ydzhang/lib/python3.6/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [47]:
train_df =pd.read_csv('invite_info.csv',index_col=0)
test_df =pd.read_csv('evalute.csv',index_col=0)

def get_day(str):
    return int(str.split('-')[0][1:])
train_df['invite_create_day'] = train_df['create_time'].apply(get_day)

ques_vecs_secs = ['title_SW_vec_his_mean', 'title_W_vec_his_mean','describe_W_vec_his_mean','title_SW_vec_his_max', 'title_W_vec_his_max','describe_W_vec_his_max']
for sec in ques_vecs_secs:
    data_df[sec] = minmax_scale(data_df[sec])

drop_columns = ['is_answer', 'question_id', 'user_id', 'topic_id','follow_topics', 'interest_topics']
train_secs = data_df.columns.drop(drop_columns)
trian_start = 3838; train_end = 3867
x_train = data_df[train_secs][:train_df.shape[0]][train_df['invite_create_day']>train_end-7].values
y_train = data_df[:train_df.shape[0]]['is_answer'][train_df['invite_create_day']>train_end-7].values
x_test = data_df[train_secs][train_df.shape[0]:].values

In [ ]:
import lightgbm as lgb
params = {
    'boosting_type':'gbdt','objective':'binary','metric':['auc', 'binary_logloss'],
    'n_jobs':-1,'num_threads':30,'n_estimators':2000,'subsample':0.8,'subsample_freq':1,
    'num_leaves':64,'learning_rate':0.01,
    'max_bin':425,'subsample_for_bin':50000,'min_split_gain':0,
    'min_child_weight':5,'min_child_samples':10,
    'colsample_bytree':1,'reg_alpha':3,'reg_lambda':5,
    'seed':1000,'silent':True    
}
lgb_train = lgb.Dataset(x_train,y_train,free_raw_data=False)#, feature_name=train_secs.to_list(), categorical_feature=category_columns)
gbm = lgb.train(params,lgb_train,valid_sets=lgb_train,early_stopping_rounds=5)

In [8]:
gbm.save_model('base_model.txt')

In [10]:
boost = lgb.Booster(model_file='base_model.txt')
y_pred = boost.predict(x_test)
test_df['is_answer'] = y_pred
test_df.to_csv('result.txt', index=False, header=False, sep='\t')

In [8]:
y_pred = gbm.predict(x_test)
test_df['is_answer'] = y_pred
test_df.to_csv('result_base.txt', index=False, header=False, sep='\t')

In [19]:
gbm.predict(x_test[0:10],raw_score=True,pred_leaf=True).shape

(10, 2000)

# DEEP

In [3]:
from deepctr.models import DeepFM,xDeepFM
from deepctr.inputs import  SparseFeat, DenseFeat,get_feature_names
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split

In [1]:
from DeepCTR_master.deepctr.models import DeepFM,xDeepFM
from DeepCTR_master.deepctr.inputs import  SparseFeat, DenseFeat,get_feature_names
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split

In [5]:
DeepFM?

Init signature:
DeepFM(
    linear_feature_columns,
    dnn_feature_columns,
    embedding_size=8,
    use_fm=True,
    dnn_hidden_units=(256, 128),
    l2_reg_linear=1e-05,
    l2_reg_embedding=1e-05,
    l2_reg_dnn=0,
    init_std=0.0001,
    seed=1024,
    dnn_dropout=0,
    dnn_activation=<function relu at 0x7fb9717f2e18>,
    dnn_use_bn=False,
    task='binary',
    device='cpu',
)
Docstring:     
Instantiates the DeepFM Network architecture.

:param linear_feature_columns: An iterable containing all the features used by linear part of the model.
:param dnn_feature_columns: An iterable containing all the features used by deep part of the model.
:param embedding_size: positive integer,sparse feature embedding_size
:param use_fm: bool,use FM part or not
:param dnn_hidden_units: list,list of positive integer or empty list, the layer number and units in each layer of DNN
:param l2_reg_linear: float. L2 regularizer strength applied to linear part
:param l2_reg_embedding: float. L2 regu

In [4]:
from DeepCTR.deepctr_torch.models import DeepFM,WDL,xDeepFM
from DeepCTR.deepctr_torch.inputs import  SparseFeat, DenseFeat,get_feature_names
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
import torch

In [5]:
boost = lgb.Booster(model_file='base_model.txt')
train_gbdt_feat = boost.predict(x_train,pred_leaf=True)

In [ ]:
np.save('train_gbdt_feat.npy',train_gbdt_feat)

In [6]:
test_gbdt_feat = boost.predict(x_test,pred_leaf=True)

In [6]:
sparse_features = [str(i) for i in range(2000)]
# sparse_features = []
# dense_features = ['question_idcount', 'salt_value', 'user_idcount', 'answer_prob', 'invite_create_hour', 'question_answer_day', 'user_answer_nums', 'invite_create_day', 'answer_create_day_user', 'create_day']
dense_pos = [train_secs.get_loc(sec) for sec in dense_features]
dense_features = []
# data_df[dense_features] = data_df[dense_features].fillna(0, )
fixlen_feature_columns = [SparseFeat(feat, 64) for feat in sparse_features] + [DenseFeat(feat, 1,) for feat in dense_features]
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(
    linear_feature_columns+ dnn_feature_columns)

train_model_input={}
for sec in sparse_features:
    train_model_input[sec]=train_gbdt_feat[:,int(sec)]
for i,sec in enumerate(dense_features):
    train_model_input[sec]=x_train[:,dense_pos[i]]

In [59]:
model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,task='binary',
               l2_reg_embedding=1e-5,device='cuda:1')
model.compile("adam", "binary_crossentropy",
              metrics=["binary_crossentropy","accuracy"],)

RuntimeError: cuda runtime error (59) : device-side assert triggered at /pytorch/aten/src/THC/generic/THCTensorMath.cu:26

In [58]:
model.fit(train_model_input, y_train,
          batch_size=256, epochs=15, validation_split=0.1, verbose=20)

KeyboardInterrupt: 

In [1]:
y_train.shape

NameError: name 'y_train' is not defined

In [20]:
# data_df = pd.read_csv('data_df.csv',index_col=0)

# category_df = member_df[['user_id', 'category_A', 'category_B','category_C', 'category_D', 'category_E','gender', 'visit_freq']]

# category_df.columns = ['user_id','category_A_raw','category_B_raw','category_C_raw','category_D_raw','category_E_raw']
# category_df.columns = [x+'_raw' if x !='user_id' else x for x in category_df.columns ]

# data_df = pd.merge(data_df,category_df,on='user_id',how='left')
drop_columns = ['is_answer', 'question_id', 'user_id', 'topic_id','follow_topics', 'interest_topics']
train_secs = data_df.columns.drop(drop_columns)
# category_columns = category_df.columns.drop('user_id')
category_columns= []
dense_features = data_df.columns.drop(category_columns).drop(drop_columns)
sparse_features = category_columns
target = ['is_answer']

In [21]:
data_df[sparse_features] = data_df[sparse_features].fillna('-1', )
data_df[dense_features] = data_df[dense_features].fillna(0, )

for feat in sparse_features:
    lbe = LabelEncoder()
    data_df[feat] = lbe.fit_transform(data_df[feat])

# 2.count #unique features for each sparse field,and record dense feature field name

fixlen_feature_columns = [SparseFeat(feat, data_df[feat].nunique())
                          for feat in sparse_features] + [DenseFeat(feat, 1,)
                                                          for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(
    linear_feature_columns+ dnn_feature_columns)

trian_start = 3838; train_end = 3867
train = data_df[train_secs.tolist()+['is_answer']][:train_df.shape[0]][train_df['invite_create_day']>train_end-7]

train_model_input = {name:train[name] for name in feature_names}

In [14]:
model = WDL(linear_feature_columns=dnn_feature_columns, dnn_feature_columns=dnn_feature_columns, task='binary',
               l2_reg_embedding=1e-5, device='cuda:1')
model.compile("adam", "binary_crossentropy",
              metrics=["binary_crossentropy", "auc"],)

NameError: name 'dnn_feature_columns' is not defined

In [23]:
model.fit(train_model_input, train[target].values,
          batch_size=512, epochs=35, validation_split=0, verbose=2)

cuda:1
Train on 2593669 samples, validate on 0 samples, 5066 steps per epoch
Epoch 1/35
100s - loss:  0.3925 - binary_crossentropy:  0.3925 - auc:  0.7277


KeyboardInterrupt: 

In [ ]:
model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns, task='binary',
               l2_reg_embedding=1e-5, device='cuda:1')

model.compile("adam", "binary_crossentropy",
              metrics=["binary_crossentropy", "auc"],)
model.fit(train_model_input, train[target].values,
          batch_size=256, epochs=50, validation_split=0, verbose=2)

test = data_df[train_secs.tolist()+['is_answer']][train_df.shape[0]:].reset_index(drop=True)

test_model_input = {name:test[name] for name in feature_names}

pred_ans = model.predict(test_model_input, 256)
print("")

# LR

In [50]:
import lightgbm as lgb

boost = lgb.Booster(model_file='base_model.txt')

train_gbdt_feat = boost.predict(x_train,pred_leaf=True)

test_gbdt_feat = boost.predict(x_test,pred_leaf=True)

y_pred = lr.predict_proba(test_gbdt_feat)

y_pred = y_pred[:,1]

from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(loss='log',tol=1e-4,penalty='l2',alpha=0.01,n_jobs=-1)

sgd.partial_fit(train_gbdt_feat,y_train)
import numpy as np
train_gbdt_feat = np.load('train_gbdt_feat.npy')
y_train = np.load('y_train.npy')

from sklearn.linear_model import LogisticRegression
lm = LogisticRegression(penalty='l2',C=0.05,n_jobs=-1,verbose=20)

sgd.partial_fit(train_gbdt_feat[0:10000],y_train[0:10000],classes=np.array([0., 1.]))

np.unique(y_train[0:100])

In [48]:
y_train[0:1000].sum()

170.0

In [3]:
train_gbdt_feat.shape

(2593669, 2000)

In [40]:
lm.fit(train_gbdt_feat[-100000:],y_train[-100000:])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 56 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 10.1min
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 10.1min finished


LogisticRegression(C=0.05, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=20,
                   warm_start=False)

In [41]:
y_pred = lm.predict_proba(train_gbdt_feat)
roc_auc_score(y_train,y_pred[:,1])

0.8243454106181634

In [ ]:
test_df['is_answer'] = warm_pred
test_df.to_csv('result_lr.txt', index=False, header=False, sep='\t')

In [7]:
warm_lm =  LogisticRegression(penalty='l2',C=0.05,n_jobs=-1,verbose=20,warm_start=True)

In [35]:
train_feats = train_gbdt_feat[0:1000000]
train_feats_y = y_train[0:1000000]

now_state = np.random.get_state()
np.random.shuffle(train_feats)
np.random.set_state(now_state)
np.random.shuffle(train_feats_y)

In [8]:
step = 100000
for i in range(0,train_gbdt_feat.shape[0],step):
    warm_lm.fit(train_gbdt_feat[i:i+step],y_train[i:i+step])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 56 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  9.6min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 56 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  9.7min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 56 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  8.9min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 56 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  8.8min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 56 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 10.1min
[Parallel(n_jobs=-1)]

In [12]:
warm_lm.intercept_,warm_lm.coef_

(array([-0.00025849]),
 array([[-0.00014527,  0.00015248, -0.00328656, ...,  0.00022596,
          0.00245392,  0.00196918]]))

In [38]:
warm_lm.intercept_,warm_lm.coef_

(array([-0.00027655]),
 array([[ 0.00726306,  0.00117516,  0.00672448, ...,  0.00335834,
         -0.00269841, -0.00351842]]))

In [50]:
lm_pred = lm.predict_proba(test_gbdt_feat)[:,1]

NameError: name 'test_gbdt_feat' is not defined

In [21]:
warm_pred = warm_lm.predict_proba(test_gbdt_feat)[:,1]

In [19]:
test_gbdt_feat = np.load('test_gbdt_feat.npy')

In [22]:
test_df['is_answer'] = warm_pred
test_df.to_csv('result_lr.txt', index=False, header=False, sep='\t')

In [36]:
from sklearn.metrics import roc_auc_score

In [25]:
roc_auc_score(y_train[-100:],lm_pred)

0.8830620855937312

In [38]:
roc_auc_score(y_train,y_pred[:,1])

0.835652043580662

In [11]:
import pickle
with open('lm_warm.pickle','wb') as f:
    pickle.dump(warm_lm,f)

NameError: name 'lm1' is not defined

In [16]:
with open('lm1.pickle','rb') as f:
    lm1 = pickle.load(f)

In [35]:
y_pred = lm1.predict_proba(train_gbdt_feat)

In [33]:
test_df['is_answer'] = y_pred[:,1]
test_df.to_csv('result_lr.txt', index=False, header=False, sep='\t')

In [34]:
y_pred

array([[0.68189172, 0.31810828],
       [0.99704143, 0.00295857],
       [0.8947785 , 0.1052215 ],
       ...,
       [0.52821261, 0.47178739],
       [0.96166116, 0.03833884],
       [0.94026906, 0.05973094]])